# 篩選十年來始終存在的股票

In [5]:
# To identify stock codes (股票代碼) that appear consistently across a decade of stock information files.
import pandas as pd
import os

# Set the current working directory to where the stock information files are stored
file_path = r"C:\Users\Sydney\Desktop\KCL\Undergraduate\Year3\Final Individual Project\Data_Collection\Status_of_TWSE"
os.chdir(file_path)

# Generate a list of file names 
file_names = [f"{year}{month:02d}.csv" for year in range(2010, 2020) for month in range(1, 13)]
# print(file_names)

# Initialize an empty DataFrame to hold all stock data
all_stocks = pd.DataFrame()


# Loop through each file name and concatenate its data to the all_stocks DataFrame
for file_name in file_names:
    #print(file_name)
    df = pd.read_csv(file_name)
    # Concatenate the current file's data
    all_stocks = pd.concat([all_stocks, df], ignore_index = True)
    #print(all_stocks)

# Check if the all_stocks DataFrame is not empty
if not all_stocks.empty:
    # Identify stocks that appear in all files by counting occurrences 
    # and checking if equal to the number of files
    common_stocks = all_stocks['股票代碼'].value_counts() == len(file_names) # '股票代碼' means 'stock code'
    # Filter to keep only stocks present in all files
    common_stocks = common_stocks[common_stocks].index.tolist()
    print("Stock codes present in all files: ", common_stocks)
    print("Total number of common stocks: ", len(common_stocks))
    
else:
    print("No common stock codes found")   

Stock codes present in all files:  ['2489', '3406', '2448', '2466', '2486', '3059', '3050', '3031', '3049', '3008', '3019', '2426', '3038', '3024', '2499', '2406', '2409', '6206', '3060', '3231', '3494', '3515', '6117', '6128', '6166', '6235', '3504', '6277', '9912', '2323', '2340', '2349', '2374', '2393', '3481', '3557', '3535', '3062', '2450', '2455', '2485', '2498', '3025', '3027', '3045', '3047', '3311', '2439', '3380', '3419', '3596', '4904', '4906', '5388', '6136', '6142', '2444', '2419', '3054', '6176', '3576', '3593', '3622', '5305', '5484', '6120', '6164', '6168', '6209', '2412', '6226', '1102', '8072', '8105', '2314', '2332', '2345', '2402', '3057', '3002', '3022', '3017', '2451', '2454', '2458', '2481', '3006', '3014', '3016', '3034', '3035', '3041', '3056', '3094', '3189', '3443', '3532', '3545', '3588', '2449', '2441', '2436', '2329', '9918', '9926', '9931', '9937', '1437', '2302', '2303', '2330', '2425', '2338', '2344', '2351', '2363', '2369', '2379', '2401', '5471', '620

# 以十年平均成交量篩選股票 = 流動性

In [6]:
# To calculate the average trading volume over a decade for a set of common stocks, identifying their liquidity.
def calculate_average_trading_volume(df):
    # Create a copy of the DataFrame to avoid modifying the original data
    df = df.copy()
    
    # Convert trading volume to numeric, coercing errors to NaN
    df['本月成交量(千股)'] = pd.to_numeric(df['本月成交量(千股)'], errors='coerce')
    # Calculate the mean of the monthly trading volumes
    average_volume = df['本月成交量(千股)'].mean()
    
    return average_volume
# Change the current working directory to where the stock information files are stored
file_path = r"C:\Users\Sydney\Desktop\KCL\Undergraduate\Year3\Final Individual Project\Data_Collection\Status_of_TWSE"
os.chdir(file_path)

# Generate a list of filenames
file_names = [f"{year}{month:02d}.csv" for year in range(2010, 2020) for month in range(1, 13)]

# Initialize a list to store the average volumes for each month
monthly_average_volumes = []

for file_name in file_names:
    df = pd.read_csv(file_name)
    
    # Filter the DataFrame to include only rows with stock codes present in the common_stocks list
    df_filtered = df[df['股票代碼'].isin(common_stocks)]
    
    # If the filtered DataFrame is not empty, calculate the monthly average volume
    if not df_filtered.empty:
        monthly_avg_vol = calculate_average_trading_volume(df_filtered)
        
        monthly_average_volumes.append(monthly_avg_vol)

# Calculate the overall average trading volume across all months
average_trading_volume = sum(monthly_average_volumes)/len(monthly_average_volumes)

print(f"The average of all monthly average trading volumes is: {average_trading_volume}")

The average of all monthly average trading volumes is: 57342.92360467928


# 計算預期回報 Expected Return Value

In [7]:
# To calculate the expected returns for each stock based on their monthly price changes, 
# and saves the updated DataFrame with an added column for expected returns.

# Function to calculate the expected return of stocks
# return type: float (in decimal form)
def calculate_expected_return(df):
    df = df.copy()
    
    # Initialize the 'expected return' column
    df['預期回報'] = None
    
    # Inner function to calculate expected return for each row
    def calculate_row_expected_return(row):
        try:
            # Convert current and previous month prices to float, handle '-' as None
            current_month_last_price = float(row['本月底市價(元)']) if row['本月底市價(元)'] != '-' else None
            previous_month_last_price = float(row['上月底市價(元)']) if row['上月底市價(元)'] != '-' else None

            # Calculate return if both prices are available
            if current_month_last_price is not None and previous_month_last_price is not None:
                return (current_month_last_price - previous_month_last_price) / previous_month_last_price
        except ValueError:
            pass
        # Return None if unable to calculate
        return None

    # Apply the inner function to each row to calculate expected returns
    df['預期回報'] = df.apply(calculate_row_expected_return, axis=1)
    
    return df

for file_name in file_names:
    df = pd.read_csv(file_name)
    
    # Filter DataFrame for stocks present in the common stocks list
    df_filtered = df[df['股票代碼'].isin(common_stocks)]
    
    if not df_filtered.empty:
        # Calculate the expected return for the filtered DataFrame
        df_with_return = calculate_expected_return(df_filtered)      
        print(f"Processed {file_name}")
        
        # Define the name for the result file
        result_file_name = file_name.replace('.csv', '_expected_return.csv')
        
        # Save the updated DataFrame with expected returns to a new file
        df_with_return.to_csv(result_file_name, index=False)
        print(f"Saved result to {result_file_name}")

Processed 201001.csv
Saved result to 201001_expected_return.csv
Processed 201002.csv
Saved result to 201002_expected_return.csv
Processed 201003.csv
Saved result to 201003_expected_return.csv
Processed 201004.csv
Saved result to 201004_expected_return.csv
Processed 201005.csv
Saved result to 201005_expected_return.csv
Processed 201006.csv
Saved result to 201006_expected_return.csv
Processed 201007.csv
Saved result to 201007_expected_return.csv
Processed 201008.csv
Saved result to 201008_expected_return.csv
Processed 201009.csv
Saved result to 201009_expected_return.csv
Processed 201010.csv
Saved result to 201010_expected_return.csv
Processed 201011.csv
Saved result to 201011_expected_return.csv
Processed 201012.csv
Saved result to 201012_expected_return.csv
Processed 201101.csv
Saved result to 201101_expected_return.csv
Processed 201102.csv
Saved result to 201102_expected_return.csv
Processed 201103.csv
Saved result to 201103_expected_return.csv
Processed 201104.csv
Saved result to 201

# 

# 將每一支股票120個月的數據分別儲存

In [27]:
# To store data for each stock separately over 120 months
import pandas as pd
import os

# Set the directory where the stock information files are stored
file_path = r"C:\Users\Sydney\Desktop\KCL\Undergraduate\Year3\Final Individual Project\Data_Collection\Status_of_TWSE"
os.chdir(file_path)

# Generate a list of filenames
file_names = [f"{year}{month:02d}_expected_return.csv" for year in range(2010, 2020) for month in range(1, 13)]
#print(file_names)

# Specify the columns to keep from each file
columns_to_keep = ['股票代碼', '股票名稱','本月成交量(千股)', '預期回報']

# Initialize a dictionary to store data for each stock separately
data_per_stock = {}

for file_name in file_names:
    df = pd.read_csv(file_name, usecols=columns_to_keep)
    print(f"Now processing {file_name}.")
    
    # Extract the date from the filename
    date = file_name[:6]
    # print(date)
    
    # Add the extracted date as a new column to the DataFrame
    df['日期'] = date
    
    for index, row in df.iterrows():
        stock_code = row['股票代碼']
        
        # Initialize a new DataFrame for the stock code if it doesn't exist in the dictionary
        if stock_code not in data_per_stock:
            data_per_stock[stock_code] = pd.DataFrame(columns=df.columns)
        
        # Concatenate the current row to the corresponding stock's DataFrame
        data_per_stock[stock_code] = pd.concat([data_per_stock[stock_code], pd.DataFrame([row])], ignore_index=True)
      

Now processing 201001_expected_return.csv.
201001
Now processing 201002_expected_return.csv.
201002
Now processing 201003_expected_return.csv.
201003
Now processing 201004_expected_return.csv.
201004
Now processing 201005_expected_return.csv.
201005
Now processing 201006_expected_return.csv.
201006
Now processing 201007_expected_return.csv.
201007
Now processing 201008_expected_return.csv.
201008
Now processing 201009_expected_return.csv.
201009
Now processing 201010_expected_return.csv.
201010
Now processing 201011_expected_return.csv.
201011
Now processing 201012_expected_return.csv.
201012
Now processing 201101_expected_return.csv.
201101
Now processing 201102_expected_return.csv.
201102
Now processing 201103_expected_return.csv.
201103
Now processing 201104_expected_return.csv.
201104
Now processing 201105_expected_return.csv.
201105
Now processing 201106_expected_return.csv.
201106
Now processing 201107_expected_return.csv.
201107
Now processing 201108_expected_return.csv.
201108


In [9]:
# To store data for each stock separately over 120 months
import pandas as pd
import os

# Set the directory where the stock information files are stored
file_path = r"C:\Users\Sydney\Desktop\KCL\Undergraduate\Year3\Final Individual Project\Data_Collection\Status_of_TWSE"
os.chdir(file_path)

# Generate a list of filenames
file_names = [f"{year}{month:02d}_expected_return.csv" for year in range(2010, 2020) for month in range(1, 13)]
#print(file_names)

# Specify the columns to keep from each file
columns_to_keep = ['股票代碼', '股票名稱','本月成交量(千股)', '預期回報']

# Initialize a dictionary to store data for each stock separately
data_per_stock = {}

for file_name in file_names:
    df = pd.read_csv(file_name, usecols=columns_to_keep)
    print(f"Now processing {file_name}.")
    
    # Extract the date from the filename
    date = file_name[:6]
    # print(date)
    
    # Add the extracted date as a new column to the DataFrame
    df['日期'] = date
    
    for index, row in df.iterrows():
        stock_code = row['股票代碼']
        
        # Initialize a new DataFrame for the stock code if it doesn't exist in the dictionary
        if stock_code not in data_per_stock:
            data_per_stock[stock_code] = pd.DataFrame(columns=df.columns)
        
        # Concatenate the current row to the corresponding stock's DataFrame
        data_per_stock[stock_code] = pd.concat([data_per_stock[stock_code], pd.DataFrame([row])], ignore_index=True)

# The folder path that the stock info is going to be sorted 
target_folder = r"C:\Users\Sydney\Desktop\KCL\Undergraduate\Year3\Final Individual Project\Data_Collection\Status_of_TWSE\per_stock_data"

stock_file_names = []

# Ensure the target folder exists; if not, create it
if not os.path.exists(target_folder):
    os.makedirs(target_folder)
    
else:
    # Change the working directory to the target folder
    os.chdir(target_folder)

# Iterate through the dictionary, saving a CSV file for each stock
for stock_code, df in data_per_stock.items():
    # Get the unique stock name from the DataFrame (assuming the same stock code has a consistent name)
    stock_name = df['股票名稱'].iloc[0]
    # Combine the stock code and stock name to form the filename
    output_filename = f"{stock_code}_{stock_name}.csv"
    # Replace any characters in the filename that may be invalid for file paths
    output_filename = output_filename.replace('/', '_').replace('\\', '_')
    print(f"Saving {output_filename}")
    df.to_csv(output_filename, index=False)
    print(f"{output_filename} saving success.")
    # Keep track of the names of the files saved
    stock_file_names.append(output_filename)

# print(stock_file_names)
# print(len(stock_file_names))

Now processing 201001_expected_return.csv.
Now processing 201002_expected_return.csv.
Now processing 201003_expected_return.csv.
Now processing 201004_expected_return.csv.
Now processing 201005_expected_return.csv.
Now processing 201006_expected_return.csv.
Now processing 201007_expected_return.csv.
Now processing 201008_expected_return.csv.
Now processing 201009_expected_return.csv.
Now processing 201010_expected_return.csv.
Now processing 201011_expected_return.csv.
Now processing 201012_expected_return.csv.
Now processing 201101_expected_return.csv.
Now processing 201102_expected_return.csv.
Now processing 201103_expected_return.csv.
Now processing 201104_expected_return.csv.
Now processing 201105_expected_return.csv.
Now processing 201106_expected_return.csv.
Now processing 201107_expected_return.csv.
Now processing 201108_expected_return.csv.
Now processing 201109_expected_return.csv.
Now processing 201110_expected_return.csv.
Now processing 201111_expected_return.csv.
Now process

1583_程泰 GOODWAY.csv saving success.
Saving 2049_上銀科技 HIWIN.csv
2049_上銀科技 HIWIN.csv saving success.
Saving 4526_東台 TOPPER.csv
4526_東台 TOPPER.csv saving success.
Saving 4532_瑞智 RECHI.csv
4532_瑞智 RECHI.csv saving success.
Saving 6605_帝寶 DEPO.csv
6605_帝寶 DEPO.csv saving success.
Saving 8374_羅昇 ACE PILLAR.csv
8374_羅昇 ACE PILLAR.csv saving success.
Saving 1603_中華電纜 CWCO.csv
1603_中華電纜 CWCO.csv saving success.
Saving 1605_華新麗華 WALSIN.csv
1605_華新麗華 WALSIN.csv saving success.
Saving 1608_華榮 HE.csv
1608_華榮 HE.csv saving success.
Saving 1609_大亞電纜 TA YA.csv
1609_大亞電纜 TA YA.csv saving success.
Saving 1611_中國電器 CE.csv
1611_中國電器 CE.csv saving success.
Saving 1612_宏泰電工 Hong Tai.csv
1612_宏泰電工 Hong Tai.csv saving success.
Saving 1614_三洋電機 SANYO.csv
1614_三洋電機 SANYO.csv saving success.
Saving 1615_大山電纜 Dah San.csv
1615_大山電纜 Dah San.csv saving success.
Saving 1616_億泰電纜 Evertop.csv
1616_億泰電纜 Evertop.csv saving success.
Saving 1617_榮星電線 JSW.csv
1617_榮星電線 JSW.csv saving success.
Saving 1618_合機電纜 Hold-Key.csv
1

1701_中國化學 CCPC.csv saving success.
Saving 1707_葡萄王 Grape King.csv
1707_葡萄王 Grape King.csv saving success.
Saving 1720_生達化學 S.C.P.C.csv
1720_生達化學 S.C.P.C.csv saving success.
Saving 1731_美吾華 Maywufa.csv
1731_美吾華 Maywufa.csv saving success.
Saving 1733_五鼎 ApexBio.csv
1733_五鼎 ApexBio.csv saving success.
Saving 1734_杏輝藥品 SINPHAR.csv
1734_杏輝藥品 SINPHAR.csv saving success.
Saving 1736_喬山 JHT.csv
1736_喬山 JHT.csv saving success.
Saving 4104_佳醫 EXCELSIOR.csv
4104_佳醫 EXCELSIOR.csv saving success.
Saving 4106_雃博 APEX.csv
4106_雃博 APEX.csv saving success.
Saving 4108_懷特 PhytoHealth.csv
4108_懷特 PhytoHealth.csv saving success.
Saving 4119_旭富 SCI.csv
4119_旭富 SCI.csv saving success.
Saving 4133_亞諾法 Abnova.csv
4133_亞諾法 Abnova.csv saving success.
Saving 2616_山隆 SLC.csv
2616_山隆 SLC.csv saving success.
Saving 6505_台塑化 FPCC.csv
6505_台塑化 FPCC.csv saving success.
Saving 8926_台汽電 TCC.csv
8926_台汽電 TCC.csv saving success.
Saving 9908_大台北瓦斯 GTG.csv
9908_大台北瓦斯 GTG.csv saving success.
Saving 9918_欣天然 S.S.N.G..csv
991

6141_柏承科技 Plotech.csv saving success.
Saving 6153_嘉聯益 Career Tech..csv
6153_嘉聯益 Career Tech..csv saving success.
Saving 6155_鈞寶電子 KC.csv
6155_鈞寶電子 KC.csv saving success.
Saving 6191_精成科技 GBM.csv
6191_精成科技 GBM.csv saving success.
Saving 6205_詮欣 COXOC.csv
6205_詮欣 COXOC.csv saving success.
Saving 6213_聯茂 ITEQ.csv
6213_聯茂 ITEQ.csv saving success.
Saving 6224_聚鼎科技 PTTC..csv
6224_聚鼎科技 PTTC..csv saving success.
Saving 6251_定穎電子 DYNAMIC.csv
6251_定穎電子 DYNAMIC.csv saving success.
Saving 6269_台郡 FLEXIUM.csv
6269_台郡 FLEXIUM.csv saving success.
Saving 6282_康舒 AcBel.csv
6282_康舒 AcBel.csv saving success.
Saving 8039_台虹 TAIFLEX.csv
8039_台虹 TAIFLEX.csv saving success.
Saving 8046_南電 N.P.C..csv
8046_南電 N.P.C..csv saving success.
Saving 8103_瀚荃股份有限公司 CviLux.csv
8103_瀚荃股份有限公司 CviLux.csv saving success.
Saving 8163_達方電子 DARFON.csv
8163_達方電子 DARFON.csv saving success.
Saving 8213_志超 tpt.csv
8213_志超 tpt.csv saving success.
Saving 8249_菱光科技 CREATIVE SENSOR INC..csv
8249_菱光科技 CREATIVE SENSOR INC..csv saving su

# 計算每支股票的月平均預期回報及波動率(標準差)

In [10]:
# To calculate the monthly average expected return and volatility (standard deviation) for each stock
import numpy as np
import pandas as pd

results = []

for each_stock_file in stock_file_names:
    df = pd.read_csv(each_stock_file)
    stock_code = df['股票代碼'].iloc[0] # Extract stock code
    stock_name = df['股票名稱'].iloc[0] # Extract stock name
    avg_return = df['預期回報'].mean() # Calculate the average expected monthly return
    
    # Calculate annualized expected return assuming compound interest
    annualized_return_compound = (1 + avg_return) ** 12 - 1
    
    # Calculate the standard deviation of expected returns as monthly volatility
    volatility = df['預期回報'].std()
    # Annualize the volatility by multiplying by the square root of 12
    annualized_volatility = volatility * np.sqrt(12)
    
    # Calculate the average trading volume for the stock
    avg_trading_vol = calculate_average_trading_volume(df)
    
    # Append the stock code, name, average trading volume, annualized return, and volatility to the results list
    results.append([stock_code, stock_name, avg_trading_vol, annualized_return_compound, annualized_volatility])

# Create a DataFrame from the results list with appropriate column names
results_df = pd.DataFrame(results, columns=['股票代碼', '股票名稱', '平均成交量', '平均預期回報', '波動率(標準差)'])
results_df.to_csv("MPT_algorithm_data.csv", index=False)
print("平均成交量、平均預期回報與標準差已儲存")

平均成交量、平均預期回報與標準差已儲存


# 篩選掉十年來小於平均成交量的股票以確保流動性
# 篩選掉預期回報小於零的股票以確保收益率

In [11]:
# To filter out stocks based on two criteria to ensure liquidity and a positive return rate over a decade.
import pandas as pd

file_path = r"C:\Users\Sydney\Desktop\KCL\Undergraduate\Year3\Final Individual Project\Data_Collection\Status_of_TWSE\per_stock_data\MPT_algorithm_data.csv"
df = pd.read_csv(file_path)

# Filter condition 1: Remove stocks with an average trading volume less than the given threshold
filter_condition1 = df['平均成交量'] > 57342.92360467928 #The average of all monthly average trading volumes 

# Filter condition 2: Remove stocks with an expected return less than zero
filter_condition2 = df['平均預期回報'] >= 0

# Combine the two conditions 
conbined_condition = filter_condition1 & filter_condition2

# Apply the combined condition to filter the DataFrame
filtered_df = df[conbined_condition]
# print(len(filtered_df))
# print(filtered_df['股票代碼'])

filtered_df.to_csv('Liquidity_Filtered_Stocks.csv', index=False)
print('Filtered stocks have been saved to Filtered_Stocks.csv.')

Filtered stocks have been saved to Filtered_Stocks.csv.
